# Import Libraries

In [ ]:
import sys
sys.path.append('..')
for p in sys.path:
    print(p)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
#import mapbox_vector_tile
from time import time
import operator
from collections import Counter
import json
import os
import math
import random

from tensorflow.keras import Sequential
from tensorflow.keras import layers

from shapely import geometry 
from PIL import Image, ImageDraw
from simplification.cutil import (
    simplify_coords,
    simplify_coords_idx,
    simplify_coords_vw,
    simplify_coords_vw_idx,
    simplify_coords_vwp,
)

# Define Functions

In [37]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(conn)
    except Error as e:
        print(e)
    
    return conn

def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def ScoreFormula(old_number_of_datapoints, new_number_of_datapoints, processing_time):
    return (1 - (new_number_of_datapoints / old_number_of_datapoints)) * (1 - processing_time)


def ScaleFactor(all_geometries):
    b_list = []
    
    for geometries in all_geometries:
        
        polygon = geometry.Polygon(geometries)
        centroid = np.array(polygon.centroid)
        coordinates = np.vstack(geometries)
        
        b = coordinates - centroid
        b_min = np.min(b)
        b_max = np.max(b)
        b_list.append(b_min)
        b_list.append(b_max)
        
    return np.std(b_list)
    
def Normalize_Geometry(coordinates1, scale_factor):
    polygon = geometry.Polygon(coordinates1)
    centroid = np.array(polygon.centroid)
    coordinates2 = np.vstack(coordinates1)
    
    return (coordinates2 - centroid) / scale_factor

def Add_One_Hot(normalized_geometry):
    normalized_geometry = np.insert(normalized_geometry, 2, 1, axis=1)
    normalized_geometry = np.insert(normalized_geometry, 3, 0, axis=1)
    normalized_geometry = np.insert(normalized_geometry, 4, 0, axis=1)
    normalized_geometry[len(normalized_geometry)-1,2] = 0
    normalized_geometry[len(normalized_geometry)-1,4] = 1
    
    return normalized_geometry

def Add_Zero_Padding(one_hotted_geometry, max_length):
    boundary = max_length - len(one_hotted_geometry)
    zero_matrix = np.zeros([boundary,5])
    return np.append(one_hotted_geometry, zero_matrix, axis=0)

def CreateGrid(poly, dx, dy):
    
    x_ls = []
    y_ls = []

    for a in poly:
        x_ls.append(a[0])
    for a in poly:
        y_ls.append(a[1])
        
    minx = min(x_ls)
    maxx = max(x_ls)
    miny = min(y_ls)
    maxy = max(y_ls)

    nx = int(math.ceil(abs(maxx - minx)/dx))
    ny = int(math.ceil(abs(maxy - miny)/dy))

    grid = []       
    for i in range(ny):   
        grid.append(geometry.LineString([[minx,max(maxy-dy*i,miny)], [maxx, max(maxy-dy*i,miny)]]))

    for j in range(nx):
        grid.append(geometry.LineString([[min(minx+dx*j,maxx), maxy], [min(minx+dx*j,maxx), miny]]))
    
    return grid
    
def CheckSameIntersections(poly, simplified_coords, grid, ROUNDING):
    
    original = geometry.Polygon(poly)
    simplified = geometry.Polygon(simplified_coords)

    o_ls = []
    s_ls = []
    for line in grid:
        x = original.intersection(line)
        y = simplified.intersection(line)
        if x:
            if x.geom_type == 'Point':
                o_ls.append(hash(tuple([round(x.coords[0][0],ROUNDING), round(x.coords[0][1],ROUNDING)])))
            if x.geom_type == 'LineString':
                for xy in x.coords:
                    o_ls.append(hash(tuple([round(xy[0],ROUNDING), round(xy[1],ROUNDING)])))
    
        if y:
            if y.geom_type == 'Point':
                s_ls.append(hash(tuple([round(y.coords[0][0],ROUNDING), round(y.coords[0][1],ROUNDING)])))
            if y.geom_type == 'LineString':
                for xy in y.coords:
                    s_ls.append(hash(tuple([round(xy[0],ROUNDING), round(xy[1],ROUNDING)])))
        
    return len(list(set(o_ls).intersection(s_ls))) / len(set(o_ls))

    
def alter_by_zoom(poly, zoom):

    mpp = {
    '0' : 156543,
    '1' : 78271.5,
    '2' : 39135.8,
    '3' : 19567.88,
    '4' : 9783.94,
    '5' : 4891.97,
    '6' : 2445.98,
    '7' : 1222.99,
    '8' : 611.5,
    '9' : 305.75,
    '10' : 152.87,
    '11' : 76.44,
    '12' : 38.219,
    '13' : 19.109,
    '14' : 9.555,
    '15' : 4.777,
    '16' : 2.3887,
    '17' : 1.1943,
    '18' : 0.5972,
    '19' : 0.2986,
    '20' : 0.14929,
    '21' : 0.074646,
    '22' : 0.037323
    }
    return (np.array(poly) / mpp[str(zoom)]).tolist()


def check_pixel_similarity(original_coords, simplified_coords, zoom):
    
    poly1 = alter_by_zoom(original_coords, zoom)
    poly2 = alter_by_zoom(simplified_coords, zoom)

    x = []
    y = []
    for a in poly1:
        x.append(a[0])
        y.append(a[1])
    
    for a in poly1:
        a[0] = a[0] - min(x)
        a[1] = a[1] - min(y)
    
    for a in poly2:
        a[0] = a[0] - min(x)
        a[1] = a[1] - min(y)
    
    width = int(max(x) - min(x))
    height = int(max(y) - min(y))

    poly1 = [tuple(x) for x in poly1]
    poly2 = [tuple(x) for x in poly2]

    img1 = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img1).polygon(poly1, outline=1, fill=0)
    mask1 = np.array(img1)
    
    img2 = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img2).polygon(poly2, outline=1, fill=0)
    mask2 = np.array(img2)
    
    return np.sum(mask1 == mask2) / (width*height)

# Load Data

## Pand Centrum

In [ ]:
conn_pand_centrum = create_connection("/Users/davemeijdam/Documents/Data Science/Master/Master Thesis/Data/SQLite/Pand_26116_centrum.db")

cur = conn_pand_centrum.cursor()
cur.execute("SELECT data FROM tiles;")

rows = cur.fetchall()
pand_centrum_data = []
for row in rows:
    pand_centrum_data.append(mapbox_vector_tile.decode(row[0]))
    #print(row[0])
print(len(pand_centrum_data))

## Wegdeel Buiten

In [ ]:
conn_wegdeel_buiten = create_connection("/Users/davemeijdam/Documents/Data Science/Master/Master Thesis/Data/SQLite/Wegdeel_23770_buitengebied.db")

cur = conn_wegdeel_buiten.cursor()
cur.execute("SELECT data FROM tiles;")

rows = cur.fetchall()
wegdeel_buiten_data = []
for row in rows:
    wegdeel_buiten_data.append(mapbox_vector_tile.decode(row[0]))

In [3]:
path = '/Users/davemeijdam/Documents/Data Science/Master/Master Thesis/Data/Sample_data_03_05/'
Polygons = []
Types = []

for filename in os.listdir(path):
    if "geometrie." in filename:
        print(filename)
        
        f = open(str(path + filename))
        jsondata = json.load(f)
        
        

        for a in jsondata['features']:
            if len(a['geometry']['coordinates']) == 1:
                Polygons.append(a['geometry']['coordinates'][0])
                Types.append(a['geometry']['type'])
            if a['geometry']['type'] == 'LineString':
                Polygons.append(a['geometry']['coordinates'])
                Types.append(a['geometry']['type'])
            else:
                for b in a['geometry']['coordinates']:
                    Polygons.append(b)
                    Types.append(a['geometry']['type'])
            
geometry_df = pd.DataFrame({'geometry':Polygons,
                            'type':Types})
    
    
    

#f = open('/Users/davemeijdam/Documents/Data Science/Master/Master Thesis/Data/Sample_data_03_05/spoor_export_buitengebied_geometrie.json')
#wegdeeljson = json.load(f)
#wegdeeljson



waterdeel_export_stedelijk_geometrie.json
wegdeel_export_buitengebied_geometrie.json
bag_pand_buitengebeid_export_geometrie.json
wegdeel_export_stedelijk_geometrie.json
spoor_export_stedelijk_geometrie.json
waterdeel_export_buitengebied_geometrie.json
bag_pand_stedelijk_export_geometrie.json
spoor_export_buitengebied_geometrie.json


In [ ]:
wegdeeljson['features'][:5]


In [ ]:
wegdeeljson['features'][0]['geometry']['type']

In [ ]:
import shapely.geometry as sg
import shapely.ops as so
import matplotlib.pyplot as plt

ls = []
#for a in wegdeeljson['features'][:5]:
#    ls.append(geometry.Polygon(a['geometry']['coordinates'][0]))

new_shape = so.cascaded_union(ls)
fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')

for geom in new_shape.geoms:    
    xs, ys = geom.exterior.xy    
    axs.fill(xs, ys, alpha=1, fc='r', ec='none')

plt.show()

In [ ]:
import shapely.geometry as sg
import shapely.ops as so
import matplotlib.pyplot as plt


ls = []
for element in wegdeel_buiten_data[3]['wegdeel.se_fld15_vlakgeometrie2d']['features']:
    
    #print(element['geometry']['coordinates'][0])
    #geometry.Polygon(element['geometry']['coordinates'][0])
    element2 = element['geometry']
    
    if element2['type'] == 'MultiPolygon':
        if element2['coordinates']:
            for poly in element2['coordinates'][0]:
                print(poly)
                ls.append(geometry.Polygon(poly))
    
    else:
        ls.append(geometry.Polygon(element['geometry']['coordinates'][0]))

#r1 = sg.Polygon([[243, 2760], [242, 2760], [242, 2761], [243, 2760]])
#r2 = sg.Polygon([[243, 2759], [243, 2760], [244, 2760], [244, 2759], [243, 2759]])
#r3 = sg.Polygon([[244, 2759], [243, 2759], [243, 2760], [244, 2760], [244, 2759]])
#r4 = sg.Polygon([[243, 2759], [242, 2759], [242, 2760], [243, 2760], [243, 2759]])
#r5 = sg.Polygon([[241, 2759], [241, 2760], [242, 2759], [241, 2759]])

new_shape = so.cascaded_union(ls)
fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')

for geom in new_shape.geoms:    
    xs, ys = geom.exterior.xy    
    axs.fill(xs, ys, alpha=1, fc='r', ec='none')

plt.show()

# Pre Processing

## Parameters

In [40]:
# Simplification Possibilities
simplify_possibilities = [['D-P', 0], ['D-P', 0.5], ['D-P', 0.1], ['D-P', 0.05], ['D-P', 0.01], ['D-P', 0.005], 
                          ['D-P', 0.001], ['V-W', 0.5], ['V-W', 0.1], ['V-W', 0.05], ['V-W', 0.01], 
                          ['V-W', 0.005], ['V-W', 0.001], ['V-W', 0.0005], ['V-W', 0.0001], ['V-W', 0.00005]]

# Polygon length evaluation
MAX_LENGTH_DEFICIT = -0.1

# Grid
dx = 1
dy = 1
ROUNDING = 1

MIN_INTERSECTIONS_PERC = 0.75

In [ ]:
Lines = []
Polygons = []
MultiPolygons = []
a=0
for row in pand_centrum_data[:10000]:
    print(str(a) + " / " + str(len(pand_centrum_data)), end="\r")
    a = a + 1
    keys = row.keys()
    
    for key in keys:
        for element in row[key]['features']:
            
            if element['geometry']['type'] == 'LineString': 
                Lines.append(element['geometry']['coordinates'])
            
            if element['geometry']['type'] == 'Polygon':
                Polygons.append(element['geometry']['coordinates'][0])
                
            #if element['geometry']['type'] == 'MultiPolygon':
                #MultiPolygons.append(element['geometry']['coordinates'])
    
    

#test = lvl10_data[0]['spoor.se_fld12_lijngeometrie2d']['features'][0]['geometry']['coordinates']
#print(Polygons)

In [ ]:
Lines = []
Polygons = []
MultiPolygons = []
a=0
for row in wegdeel_buiten_data:
    print(str(a) + " / " + str(len(wegdeel_buiten_data)), end="\r")
    a = a + 1
    keys = row.keys()
    
    for key in keys:
        for element in row[key]['features']:
            
            if element['geometry']['type'] == 'LineString': 
                Lines.append(element['geometry']['coordinates'])
            
            if element['geometry']['type'] == 'Polygon':
                Polygons.append(element['geometry']['coordinates'][0])
                
            if element['geometry']['type'] == 'MultiPolygon':
                if element['geometry']['coordinates']:
                    for poly in element['geometry']['coordinates'][0]:
                        MultiPolygons.append(poly)
    
    

#test = lvl10_data[0]['spoor.se_fld12_lijngeometrie2d']['features'][0]['geometry']['coordinates']
#print(Polygons)

In [ ]:
#print(len(Lines))
print(len(Polygons))
#print(len(MultiPolygons))

ls = []
for a in Polygons:
    ls.append(len(a))
    
pd.DataFrame({'lengths':Counter(ls).keys(),
              'freq':Counter(ls).values()})

In [4]:
Polygons = list(geometry_df['geometry'][geometry_df['type'] == 'Polygon'])
len(Polygons)

303244

In [41]:
results_list = []
length_list = []
Polygons_sample = random.sample(Polygons, 25000)
scale_factor = ScaleFactor(Polygons_sample)
print("Scale Factor done")


# Decide order from longest polygon to smallest polygon
for row in Polygons_sample:

    length_list.append([row, len(row)])

length_list.sort(key=operator.itemgetter(1), reverse=True)
print("Sorted the Polygons")
a=0
for element in length_list:
    print(str(a) + " / " + str(len(length_list)), end="\r")
    a = a + 1
    results_dict = {}
    poly1 = geometry.Polygon(element[0])
    results = []
    #grid = CreateGrid(element[0], dx, dy)
    
    for possibility in simplify_possibilities:
        

        if possibility[0] == 'D-P':
            # Simplification function Douglas-Peucker
            time_start = time()
            simplified_coordinates = simplify_coords(element[0], possibility[1])
            time_end = time()
            process_time = time_end - time_start

        if possibility[0] == 'V-W':
            # Simplification function Visvalingam-Whyatt
            time_start = time()
            simplified_coordinates = simplify_coords_vw(element[0], possibility[1])
            time_end = time()
            process_time = time_end - time_start
        
        
        if len(simplified_coordinates) >= 3:
            poly2 = geometry.Polygon(simplified_coordinates)
            #length_deficit = (poly2.length - poly1.length) / poly1.length
        
            # If the length deficit of the polygon is smaller(greater) than the provided MAX_LENGTH_DEFICIT, 
            # the score gets saved
            #if length_deficit > MAX_LENGTH_DEFICIT:
            
            #if length_deficit == 0:
            #    score = ScoreFormula(len(element[0]), len(simplified_coordinates), process_time)
            #    results.append(score)
            #    continue
                
            #try:
            #    if CheckSameIntersections(element[0], simplified_coordinates, grid, ROUNDING) > MIN_INTERSECTIONS_PERC:
            #        score = ScoreFormula(len(element[0]), len(simplified_coordinates), process_time)
            #        results.append(score)
            #except Exception:
            #    continue
            
            if np.isnan(check_pixel_similarity(element[0], simplified_coordinates, 17)) == True:
                results.append('Remove')
                break
                
                
            if check_pixel_similarity(element[0], simplified_coordinates, 17) == 1:
                score = ScoreFormula(len(element[0]), len(simplified_coordinates), process_time)
                results.append(score)
                
    
    results_dict['polygon'] = Add_Zero_Padding(Add_One_Hot(Normalize_Geometry(element[0], scale_factor)), len(length_list[0][0]))
    if results[0] == 'Remove':
        results_dict['algorithm'] = len(simplify_possibilities)
        
    else:    
        results_dict['algorithm'] = results.index(max(results))
        
    results_list.append(results_dict)

X = []
y = []
a = 0
for element in results_list:
    print(str(a) + " / " + str(len(length_list)), end="\r")
    a = a + 1
    
    X.append(element['polygon'])
    y.append(element['algorithm'])
X = np.array(X)
y = np.array(y)
# Calculate the deficit in the number of 
#point_deficit = len(coordinates) - len(simplified_coordinates)
#print('Point Deficit: ' + str(point_deficit) + ' out of ' + str(len(coordinates)))

#old_area = PolyArea(old_xs,old_ys)
#new_area = PolyArea(new_xs,new_ys)
#area_deficit_percentage = (new_area - old_area) / old_area
#print(area_deficit_percentage)

Scale Factor done
Sorted the Polygons


/opt/anaconda3/envs/infoviz/lib/python3.7/site-packages/ipykernel_launcher.py:173: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/anaconda3/envs/infoviz/lib/python3.7/site-packages/ipykernel_launcher.py:173: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
print(len(length_list[0][0]))
time_start = time()
Add_Zero_Padding(Add_One_Hot(Normalize_Geometry(length_list[0][0], scale_factor)), len(length_list[0][0]))
time_end = time()
print(time_end - time_start)

time_start = time()
CheckSameIntersections(length_list[0][0], simplified_coordinates, grid, ROUNDING)
time_end = time()
print(time_end - time_start)

# Data Stats

In [50]:
pd.DataFrame({'keys':list(Counter(y).keys()),
              'freq':list(Counter(y).values())})

,keys,freq
0,0,14718
1,1,3351
2,2,1208
3,4,1460
4,3,684
5,5,841
6,6,987
7,16,81
8,7,699
9,8,347


In [53]:
print(X[0])

[[ 3.44091214 -1.00739838  1.          0.          0.        ]
 [ 3.46945231 -0.94843036  1.          0.          0.        ]
 [ 3.47063644 -0.94028538  1.          0.          0.        ]
 ...
 [ 3.42329672 -1.06417278  1.          0.          0.        ]
 [ 3.43223643 -1.03582675  1.          0.          0.        ]
 [ 3.44091214 -1.00739838  0.          0.          1.        ]]


In [ ]:
# Select index of simplification possibility
INDEX = 6


possibility = simplify_possibilities[INDEX]

if possibility[0] == 'D-P':
    # Simplification function Douglas-Peucker
    simplified_coordinates = simplify_coords(coordinates, possibility[1])

if possibility[0] == 'V-W':
    # Simplification function Visvalingam-Whyatt
    simplified_coordinates = simplify_coords_vw(coordinates, possibility[1])

old_xs, old_ys = zip(*coordinates)
new_xs, new_ys = zip(*simplified_coordinates)

print(len(simplified_coordinates))
print(len(coordinates))

# Plotting

In [ ]:
plt.figure()
plt.plot(old_xs, old_ys)
plt.plot(new_xs, new_ys)
plt.show()

# Keras

In [54]:
input_shape = X[0].shape
print(input_shape)
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(3,3))

model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(len(simplify_possibilities)+1, activation='softmax'))

print(model.summary())

(1021, 5)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 1017, 32)          832       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 339, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 335, 64)           10304     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                1105      
Total params: 12,241
Trainable params: 12,241
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 400
EPOCHS = 50

history = model.fit(X,
                    y,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.2,
                    verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 61s 3ms/sample - loss: 2.3975 - accuracy: 0.4756 - val_loss: 0.7232 - val_accuracy: 0.9900
Epoch 2/50
20000/20000 [==============================] - 57s 3ms/sample - loss: 1.7916 - accuracy: 0.4884 - val_loss: 0.8143 - val_accuracy: 0.9900
Epoch 3/50
20000/20000 [==============================] - 55s 3ms/sample - loss: 1.7595 - accuracy: 0.4883 - val_loss: 0.7942 - val_accuracy: 0.9900
Epoch 4/50
20000/20000 [==============================] - 45s 2ms/sample - loss: 1.7494 - accuracy: 0.4877 - val_loss: 0.7341 - val_accuracy: 0.9900
Epoch 5/50
20000/20000 [==============================] - 53s 3ms/sample - loss: 1.7416 - accuracy: 0.4875 - val_loss: 0.7499 - val_accuracy: 0.9900
Epoch 6/50
20000/20000 [==============================] - 72s 4ms/sample - loss: 1.7357 - accuracy: 0.4870 - val_loss: 0.8042 - val_accuracy: 0.9900
Epoch 7/50
20000/20000 [==============================] -

KeyboardInterrupt: 

In [ ]:
len(X)

# PyTorch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(len(X), 32, 5)
        self.pool1 = nn.MaxPool1d(3, 3)
        self.conv2 = nn.Conv1d(61, 64, 5)
        self.pool2 = nn.AvgPool1d(64)
        self.fc1 = nn.Linear(13,13)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.softmax(self.fc1(x))
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')